In [1]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
# ukuran gambar (ikut contoh)
img_width  = 136
img_height = 102

# path folder dataset
dataset_path = './flower_images/'

# ambil nama-nama kelas dari nama folder
classes = os.listdir(dataset_path)

print("Classes:", classes)

# list untuk menampung data dan label
x = []
y = []

counter = 0

Classes: ['Lilly', 'Lotus', 'Orchid', 'Sunflower', 'Tulip']


In [3]:
# import gambar ke dalam x dan label ke dalam y

for class_index, class_name in enumerate(classes):
    class_folder = os.path.join(dataset_path, class_name)

    for img_name in os.listdir(class_folder):
        path = os.path.join(class_folder, img_name)
        
        # load gambar dan ubah ukurannya
        # (target_size = (height, width))
        img = tf.keras.preprocessing.image.load_img(
            path,
            target_size=(img_height, img_width)
        )
        
        # ubah gambar jadi array (H, W, 3)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        
        # masukkan ke list x
        x.append(img_array)
        
        # label = index kelas (0,1,2,3,4)
        y.append(class_index)
    counter += 1

print("Total images:", len(x))
print("Total labels:", len(y))

Total images: 5000
Total labels: 5000


In [4]:
# ubah list jadi numpy array
x = np.array(x, dtype='float32')
y = np.array(y)

print("x shape before norm:", x.shape)
print("y shape:", y.shape)

# normalisasi piksel 0–255 -> 0–1
x = x / 255.0

print("x min:", x.min(), "x max:", x.max())

x shape before norm: (5000, 102, 136, 3)
y shape: (5000,)
x min: 0.0 x max: 1.0


In [5]:
# split pertama: train dan "sisa" (sementara disebut test)
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=400
)

# split kedua: dari x_test lagi dibagi jadi val dan test final
x_val, x_test, y_val, y_test = train_test_split(
    x_test, y_test,
    test_size=0.2,
    random_state=400
)

print("x_train:", x_train.shape)
print("x_val  :", x_val.shape)
print("x_test :", x_test.shape)

num_classes = 5  # Lily, Lotus, Orchid, Sunflower, Tulip

# ubah label integer -> one-hot
y_train = to_categorical(y_train, num_classes=num_classes)
y_val   = to_categorical(y_val,   num_classes=num_classes)
y_test  = to_categorical(y_test,  num_classes=num_classes)

print("y_train:", y_train.shape)
print("y_val  :", y_val.shape)
print("y_test :", y_test.shape)


x_train: (4000, 102, 136, 3)
x_val  : (800, 102, 136, 3)
x_test : (200, 102, 136, 3)
y_train: (4000, 5)
y_val  : (800, 5)
y_test : (200, 5)


# CNN1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    # Conv layer 1
    Conv2D(
        6, (4, 4),
        activation='relu',
        input_shape=(img_height, img_width, 3)
    ),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Conv layer 2
    Conv2D(12, (4, 4), activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Flatten + Dense
    Flatten(),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),

    # output layer: 5 kelas bunga
    Dense(num_classes, activation='softmax') 
])

# compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 99, 133, 6)        294       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 66, 6)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 63, 12)        1164      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 31, 12)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 8556)              0         
                                                                 
 dense (Dense)               (None, 64)                5

In [7]:
history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    batch_size=120,
    epochs=15
)

Epoch 1/15
34/34 [==============================] - 4s 46ms/step - loss: 1.4004 - accuracy: 0.3795 - val_loss: 1.3073 - val_accuracy: 0.4850
Epoch 2/15
34/34 [==============================] - 1s 38ms/step - loss: 1.1680 - accuracy: 0.5282 - val_loss: 1.2079 - val_accuracy: 0.4762
Epoch 3/15
34/34 [==============================] - 1s 34ms/step - loss: 1.0083 - accuracy: 0.6003 - val_loss: 1.0754 - val_accuracy: 0.5700
Epoch 4/15
34/34 [==============================] - 1s 25ms/step - loss: 0.8755 - accuracy: 0.6655 - val_loss: 0.9878 - val_accuracy: 0.6025
Epoch 5/15
34/34 [==============================] - 1s 25ms/step - loss: 0.7250 - accuracy: 0.7297 - val_loss: 0.9141 - val_accuracy: 0.6550
Epoch 6/15
34/34 [==============================] - 1s 25ms/step - loss: 0.6310 - accuracy: 0.7757 - val_loss: 0.8969 - val_accuracy: 0.6612
Epoch 7/15
34/34 [==============================] - 1s 26ms/step - loss: 0.5155 - accuracy: 0.8232 - val_loss: 0.7841 - val_accuracy: 0.6963
Epoch 8/15
34

In [8]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print("Test loss :", test_loss)
print("Test acc  :", test_acc)


Test loss : 0.8127318620681763
Test acc  : 0.8100000023841858


# CNN2

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# CNN level 2
model2 = Sequential([
    # Conv block 1
    Conv2D(
        16, (3, 3),
        activation='relu',
        padding='same',
        input_shape=(img_height, img_width, 3)
    ),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Conv block 2
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Conv block 3
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Flatten + Dense
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), 

    # output layer: 5 kelas bunga
    Dense(num_classes, activation='softmax')
])

model2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 102, 136, 16)      448       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 51, 68, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 51, 68, 32)        4640      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 25, 34, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 25, 34, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 12, 17, 64)      

In [10]:
history2 = model2.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    batch_size=120,
    epochs=15
)

Epoch 1/15
34/34 [==============================] - 5s 127ms/step - loss: 1.5139 - accuracy: 0.3250 - val_loss: 1.3137 - val_accuracy: 0.4688
Epoch 2/15
34/34 [==============================] - 4s 105ms/step - loss: 1.2484 - accuracy: 0.4742 - val_loss: 1.1432 - val_accuracy: 0.5412
Epoch 3/15
34/34 [==============================] - 3s 94ms/step - loss: 1.1188 - accuracy: 0.5450 - val_loss: 1.0608 - val_accuracy: 0.5913
Epoch 4/15
34/34 [==============================] - 3s 98ms/step - loss: 1.0064 - accuracy: 0.5993 - val_loss: 0.9783 - val_accuracy: 0.6100
Epoch 5/15
34/34 [==============================] - 5s 144ms/step - loss: 0.9439 - accuracy: 0.6332 - val_loss: 0.9642 - val_accuracy: 0.6150
Epoch 6/15
34/34 [==============================] - 2s 58ms/step - loss: 0.8521 - accuracy: 0.6800 - val_loss: 0.8646 - val_accuracy: 0.6562
Epoch 7/15
34/34 [==============================] - 3s 94ms/step - loss: 0.7577 - accuracy: 0.7097 - val_loss: 0.8808 - val_accuracy: 0.6475
Epoch 8/15

In [11]:
test_loss2, test_acc2 = model2.evaluate(x_test, y_test, verbose=0)
print("Test loss (model2):", test_loss2)
print("Test acc  (model2):", test_acc2)

Test loss (model2): 0.5529642105102539
Test acc  (model2): 0.8349999785423279


# CNN3

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# CNN v3
model3 = Sequential([
    # Conv block 1
    Conv2D(
        16, (3, 3),
        activation='relu',
        padding='same',
        input_shape=(img_height, img_width, 3)
    ),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Conv block 2
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Conv block 3
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Conv block 4 (tambahan dibanding v2)
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Flatten + Dense
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),           
    
    # output layer
    Dense(num_classes, activation='softmax')
])

model3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 102, 136, 16)      448       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 51, 68, 16)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 51, 68, 32)        4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 25, 34, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 25, 34, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 12, 17, 64)      

In [14]:
history3 = model3.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    batch_size=120,
    epochs=15
)

Epoch 1/15
34/34 [==============================] - 5s 127ms/step - loss: 1.4252 - accuracy: 0.3860 - val_loss: 1.2446 - val_accuracy: 0.5238
Epoch 2/15
34/34 [==============================] - 2s 69ms/step - loss: 1.1576 - accuracy: 0.5307 - val_loss: 1.2336 - val_accuracy: 0.4950
Epoch 3/15
34/34 [==============================] - 3s 95ms/step - loss: 1.0552 - accuracy: 0.5830 - val_loss: 1.0352 - val_accuracy: 0.5900
Epoch 4/15
34/34 [==============================] - 3s 84ms/step - loss: 1.0168 - accuracy: 0.5970 - val_loss: 0.9574 - val_accuracy: 0.6438
Epoch 5/15
34/34 [==============================] - 3s 80ms/step - loss: 0.8927 - accuracy: 0.6585 - val_loss: 0.8923 - val_accuracy: 0.6513
Epoch 6/15
34/34 [==============================] - 3s 91ms/step - loss: 0.8337 - accuracy: 0.6810 - val_loss: 0.8694 - val_accuracy: 0.6750
Epoch 7/15
34/34 [==============================] - 4s 106ms/step - loss: 0.7381 - accuracy: 0.7203 - val_loss: 0.8479 - val_accuracy: 0.6675
Epoch 8/15


In [15]:
test_loss3, test_acc3 = model3.evaluate(x_test, y_test, verbose=0)
print("Test loss (model3):", test_loss3)
print("Test acc  (model3):", test_acc3)


Test loss (model3): 0.6183311343193054
Test acc  (model3): 0.8100000023841858
